.. _null-data:

# Null Data

"Never tell a lie" is an integral part of :ref:`ethos` - and Toyplot's handling of null data is one of the ways that we honor it.  Consider the following data, in which several datums contain floating-point [NaN](https://en.wikipedia.org/wiki/NaN) values:

In [1]:
import numpy
x = numpy.linspace(0, 2 * numpy.pi)
y = numpy.sin(x)
y[6:20] = numpy.nan

When we plot this data, Toyplot carefully takes the NaN values into account:

In [2]:
import toyplot.data
toyplot.plot(x, y, ymax=1, marker="o", width=600, height=300);

<path d="M 50.0 150.0 L 60.204081632653057 137.2122838315494 L 70.408163265306115 124.63454160904925 L 80.612244897959187 112.4732995120626 L 90.816326530612244 100.92824479960623 L 101.0204081632653 90.188946950878403 M 254.08163265306121 95.446509878945093 L 264.28571428571428 106.61162608824417 L 274.48979591836735 118.48917819763791 L 284.69387755102036 130.88413712986275 L 294.89795918367344 143.59297800192866 L 305.10204081632656 156.40702199807123 L 315.30612244897958 169.11586287013719 L 325.51020408163265 181.51082180236202 L 335.71428571428572 193.38837391175579 L 345.91836734693879 204.55349012105484 L 356.12244897959187 214.82283953077882 L 366.32653061224488 224.02779970753153 L 376.5306122448979 232.01722545969557 L 386.73469387755097 238.65993063730002 L 396.93877551020404 243.84684220497604 L 407.14285714285717 247.49279121818236 L 417.34693877551018 249.53791129491984 L 427.55102040816325 249.94862162006879 L 437.75510204081627 248.71817834144503 L 447.9591836734694 245.86678530366606 L 458.16326530612241 241.44126230158125 L 468.36734693877548 235.51427630053465 L 478.57142857142856 228.183148246803 L 488.77551020408157 219.56825506034869 L 498.97959183673464 209.81105304912163 L 509.18367346938771 199.07175520039385 L 519.38775510204073 187.52670048793749 L 529.59183673469374 175.36545839095083 L 539.79591836734687 162.78771616845069 L 550.0 150.00000000000003" style="stroke:rgb(40%,76.1%,64.7%);stroke-opacity:1.0;stroke-width:2.0"> 0 2 4 6 -1.0 -0.5 0.0 0.5 1.0 
 
 
 Save as .csv

Note that the Y axis domain reflects the lack of data where there are NaN values, and there are no markers for the NaN datams.  Note too that the plot has been broken into two segments - drawing a segment through the NaN region might mislead viewers about the shape of the curve, while breaking the plot unambiguously communicates the absence of data.

Of course NaN values can only be used with floating-point arrays, so there must be alternate ways to represent null values for other data types such as integers.  To address this, Toyplot uses [masked arrays](http://docs.scipy.org/doc/numpy/reference/maskedarray.html) for all its internal data structures, and accepts masked arrays for its inputs, allowing you to define null values in your data explicitly:

In [3]:
numpy.random.seed(1234)
y = numpy.ma.array(numpy.random.choice(numpy.arange(3, 10), size=50))
y[5:15] = numpy.ma.masked
toyplot.bars(y, width=600, height=300);

0 10 20 30 40 50 0 3 6 9 
 
 
 Save as .csv

You might feel that masking null values in the above example is needlessly complex, when a special value of "zero" could accomplish the same thing.  But consider what happens if there is more than one series:   

In [4]:
magnitudes = numpy.ma.column_stack((
        numpy.random.choice(numpy.arange(5, 10), size=50),
        numpy.random.choice(numpy.arange(5, 10), size=50),
    ))
magnitudes[5:15,0] = 0
toyplot.bars(magnitudes, width=600, height=300);

0 10 20 30 40 50 0 5 10 15 20 
 
 
 Save as .csv

The position of the bars in the second series suggest that the null values in the first series actually have a value of zero, when in reality we want to communicate that they have no value at all.  Contrast this with what Toyplot produces when you correctly mark the values as null instead of zero:

In [5]:
magnitudes[5:15,0] = numpy.ma.masked
toyplot.bars(magnitudes, width=600, height=300);

0 10 20 30 40 50 0 5 10 15 20 
 
 
 Save as .csv

Toyplot now removes entire observations that contain null values.  Note that this behavior is dictated by the structure of the visualization - because we use stacked bars to represent data where the sum of the magnitudes is significant, a null anywhere in that sum makes the entire sum null and void.

This is not the case for all visualizations, of course.  Consider what happens when rendering a set of bar *boundaries*, rather than a set of bar magnitudes:

In [6]:
observations = numpy.random.normal(size=(50, 50))
boundaries = numpy.ma.column_stack((
    numpy.min(observations, axis=1),
    numpy.median(observations, axis=1),
    numpy.max(observations, axis=1),
    ))

toyplot.bars(boundaries, baseline=None, width=600, height=300);

0 10 20 30 40 50 -4 -2 0 2 4 
 
 
 Save as .csv

Now, suppose that some of the lower boundaries in the plot are null:

In [7]:
boundaries[5:10, 0] = numpy.ma.masked
toyplot.bars(boundaries, baseline=None, width=600, height=300);

0 10 20 30 40 50 -4 -2 0 2 4 
 
 
 Save as .csv

In this case, the position of each bar is defined by two boundaries. Only those bars with missing boundaries are left out - the adjacent bars are still visible because they are still unambigously well-defined.  The same would be true if some of the top boundary values were null:

In [8]:
boundaries[40:45, 2] = numpy.ma.masked
toyplot.bars(boundaries, baseline=None, width=600, height=300);

0 10 20 30 40 50 -4 -2 0 2 4 
 
 
 Save as .csv

Finally, as you might imagine, null values in the middle boundary affect both sets of adjacent bars:

In [9]:
boundaries[20:30, 1] = numpy.ma.masked
toyplot.bars(boundaries, baseline=None, width=600, height=300);

0 10 20 30 40 50 -4 -2 0 2 4 
 
 
 Save as .csv

Of course, these behaviors extended to other plot types too:

In [10]:
toyplot.fill(magnitudes, baseline="stacked", width=600, height=300);  

<polygon points="200.0,250.0 210.0,250.0 220.0,250.0 230.0,250.0 240.0,250.0 250.0,250.0 260.0,250.0 270.0,250.0 280.0,250.0 290.0,250.0 300.0,250.0 310.0,250.0 320.0,250.0 330.00000000000006,250.0 340.0,250.0 350.0,250.0 360.0,250.0 370.0,250.0 380.0,250.0 390.0,250.0 400.0,250.0 410.0,250.0 420.0,250.0 430.0,250.0 440.0,250.0 450.0,250.0 460.0,250.0 470.0,250.0 480.0,250.0 490.0,250.0 500.0,250.0 510.0,250.0 520.0,250.0 530.0,250.0 540.0,250.0 540.0,160.0 530.0,190.0 520.0,190.0 510.0,180.0 500.0,190.0 490.0,180.0 480.0,190.0 470.0,180.0 460.0,190.0 450.0,200.0 440.0,190.0 430.0,160.0 420.0,190.0 410.0,180.0 400.0,170.0 390.0,190.0 380.0,180.0 370.0,160.0 360.0,160.0 350.0,180.0 340.0,180.0 330.00000000000006,160.0 320.0,170.0 310.0,160.0 300.0,180.0 290.0,180.0 280.0,190.0 270.0,170.0 260.0,180.0 250.0,160.0 240.0,200.0 230.0,160.0 220.0,200.0 210.0,200.0 200.0,190.0" style="fill:rgb(40%,76.1%,64.7%);fill-opacity:1.0;opacity:1.0;stroke:none"> <polygon points="200.0,190.0 210.0,200.0 220.0,200.0 230.0,160.0 240.0,200.0 250.0,160.0 260.0,180.0 270.0,170.0 280.0,190.0 290.0,180.0 300.0,180.0 310.0,160.0 320.0,170.0 330.00000000000006,160.0 340.0,180.0 350.0,180.0 360.0,160.0 370.0,160.0 380.0,180.0 390.0,190.0 400.0,170.0 410.0,180.0 420.0,190.0 430.0,160.0 440.0,190.0 450.0,200.0 460.0,190.0 470.0,180.0 480.0,190.0 490.0,180.0 500.0,190.0 510.0,180.0 520.0,190.0 530.0,190.0 540.0,160.0 540.0,70.0 530.0,110.00000000000001 520.0,140.0 510.0,130.0 500.0,130.0 490.0,89.999999999999986 480.0,120.0 470.0,100.0 460.0,100.0 450.0,130.0 440.0,120.0 430.0,100.0 420.0,130.0 410.0,130.0 400.0,80.0 390.0,130.0 380.0,110.00000000000001 370.0,80.0 360.0,89.999999999999986 350.0,110.00000000000001 340.0,100.0 330.00000000000006,110.00000000000001 320.0,100.0 310.0,110.00000000000001 300.0,120.0 290.0,130.0 280.0,130.0 270.0,80.0 260.0,130.0 250.0,70.0 240.0,150.0 230.0,110.00000000000001 220.0,110.00000000000001 210.0,120.0 200.0,100.0" style="fill:rgb(98.8%,55.3%,38.4%);fill-opacity:1.0;opacity:1.0;stroke:none"> 0 10 20 30 40 50 0 5 10 15 20 
 
 
 Save as .csv

In [11]:
toyplot.fill(boundaries, width=600, height=300);

<polygon points="150.0,192.06305742135285 160.0,205.33446843893492 170.0,229.53901601442837 180.0,217.15683515247051 190.00000000000003,212.57982306639266 200.0,224.31733562882386 210.0,225.76548388862935 220.0,204.02074615463974 230.0,195.96440084295554 240.0,196.16412019256387 240.0,149.40874000036459 230.0,151.42358500634865 220.0,148.36565134014185 210.0,152.75768921256673 200.0,151.96731850142154 190.00000000000003,149.56568474720609 180.0,158.25125202518771 170.0,144.48268117831972 160.0,152.77336244825773 150.0,145.73613277023685" style="fill:rgb(40%,76.1%,64.7%);fill-opacity:1.0;opacity:1.0;stroke:none"> <polygon points="350.0,197.60750552215347 360.0,239.96574959398649 370.0,205.41567879296241 380.0,210.72676230655523 390.0,215.53513441883177 400.0,206.7855343594135 410.0,193.24984914020916 420.0,181.97395381380994 430.0,216.96320312640711 440.0,212.83094312875534 450.0,193.36322495037533 460.0,201.73897945029509 470.0,213.90784218808423 480.0,208.43926980525069 490.0,199.44398336307651 500.0,202.52319796834436 510.0,214.95886884526402 520.0,201.9093946780296 530.0,211.60884582690267 540.0,202.375239100822 540.0,148.90130937567758 530.0,153.56694361419298 520.0,138.8565177528873 510.0,146.07475227024108 500.0,152.81036022600333 490.0,144.07369604155983 480.0,150.41824749296862 470.0,152.78582053956825 460.0,150.11992605411029 450.0,144.94807917207945 440.0,152.65667753703949 430.0,143.0046745701481 420.0,146.00676016871046 410.0,142.13800577291889 400.0,155.50521778259136 390.0,143.34025384104271 380.0,150.23581651253929 370.0,154.07825401133579 360.0,146.43441355873284 350.0,148.58039094762415" style="fill:rgb(40%,76.1%,64.7%);fill-opacity:1.0;opacity:1.0;stroke:none"> <polygon points="50.0,148.54314610623697 60.0,149.50980091535425 70.0,148.3310372360304 80.0,143.2753294000332 90.0,149.42840495360269 100.0,149.62855380632965 110.0,150.12478743679981 120.00000000000001,157.10481245860703 130.0,150.28023201320724 140.0,149.59652222781304 150.0,145.73613277023685 160.0,152.77336244825773 170.0,144.48268117831972 180.0,158.25125202518771 190.00000000000003,149.56568474720609 200.0,151.96731850142154 210.0,152.75768921256673 220.0,148.36565134014185 230.0,151.42358500634865 240.0,149.40874000036459 240.0,77.770748292164413 230.0,101.3691647834724 220.0,99.789131023199843 210.0,87.862023861534013 200.0,65.201127975108889 190.00000000000003,99.02802436337987 180.0,107.38263672238455 170.0,95.599179600495688 160.0,77.363734709110503 150.0,94.021298222508861 140.0,92.630760124341634 130.0,89.001296138706465 120.00000000000001,92.471220181496804 110.0,107.48123171638461 100.0,97.005283269207339 90.0,111.83095948225042 80.0,86.255840208563058 70.0,98.308217014121979 60.0,106.12754945965474 50.0,117.14544619467785" style="fill:rgb(98.8%,55.3%,38.4%);fill-opacity:1.0;opacity:1.0;stroke:none"> <polygon points="350.0,148.58039094762415 360.0,146.43441355873284 370.0,154.07825401133579 380.0,150.23581651253929 390.0,143.34025384104271 400.0,155.50521778259136 410.0,142.13800577291889 420.0,146.00676016871046 430.0,143.0046745701481 440.0,152.65667753703949 440.0,91.854129627931826 430.0,100.28098767019441 420.0,93.525215249652092 410.0,81.299910471180809 400.0,101.86126703064076 390.0,109.74939141632241 380.0,95.55952153909999 370.0,93.892867855919036 360.0,82.140712000645649 350.0,115.66040743892148" style="fill:rgb(98.8%,55.3%,38.4%);fill-opacity:1.0;opacity:1.0;stroke:none"> 0 10 20 30 40 50 -4 -2 0 2 4 
 
 
 Save as .csv

Finally, a special-case worth mentioning is Toyplot table visualizations, which *can* make an explicit distinction between null and NaN values:

In [12]:
data = toyplot.data.Table()
data["a"] = numpy.random.random(11)
data["b"] = numpy.random.random(11)
data["a", 3] = numpy.ma.masked
data["b", 7] = numpy.nan
toyplot.table(data, width=300, height=350);

a b 0 . 854699 0 . 703196 0 . 652144 0 . 282864 0 . 351356 0 . 267843 0 . 290331 0 . 525642 0 . 252136 0 . 688357 0 . 554338 0 . 00359724 0 . 24377 0 . 464203 nan 0 . 36197 0 . 437727 0 . 768892 0 . 629987 0 . 64803 0 . 0805395 
 
 
 Save as .csv

If you would rather not make this distinction, you can specify a table formatter object that will treat NaN and null values the same:

In [13]:
canvas, table = toyplot.table(data, width=300, height=350)
table.cells.column[1].format = toyplot.format.FloatFormatter(nanshow=False)

a b 0 . 854699 0 . 703196 0 . 652144 0 . 282864 0 . 351356 0 . 267843 0 . 290331 0 . 525642 0 . 252136 0 . 688357 0 . 554338 0 . 00359724 0 . 24377 0 . 464203 0 . 36197 0 . 437727 0 . 768892 0 . 629987 0 . 64803 0 . 0805395 
 
 
 Save as .csv